# Industrial Pollution Intelligence & Analysis

## Comprehensive Scientific Analysis of Water Quality Data

This notebook presents a rigorous scientific analysis of industrial pollution patterns using advanced machine learning techniques, causal inference, and product lifecycle assessment. The analysis leverages real-time water quality monitoring data from China's National Environmental Monitoring Center (CNEMC) to provide actionable insights for environmental policy and industrial management.

### Key Innovations:
- **Multi-dimensional Analysis**: PCA, t-SNE, and UMAP for pattern discovery
- **Causal Inference**: Granger causality testing for pollution relationships
- **Product Lifecycle Tracking**: Smartphone manufacturing pollution correlation
- **Advanced Forecasting**: LSTM, Prophet, and ensemble models
- **Network Analysis**: Pollution propagation and community detection


In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Scientific computing
from scipy import stats
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.ensemble import IsolationForest
from sklearn.metrics import silhouette_score

# Time series analysis
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller

# Machine learning
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Network analysis
import networkx as nx

# Set style for plots
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("Libraries imported successfully")
print(f"TensorFlow version: {tf.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")


## 1. Data Loading and Preprocessing

### 1.1 Load Processed Water Quality Data

The dataset contains 4-hour interval water quality measurements from monitoring stations across China, including:
- **Physical parameters**: Temperature, pH, dissolved oxygen, conductivity, turbidity
- **Chemical parameters**: Ammonia nitrogen, total phosphorus, total nitrogen, permanganate index
- **Biological parameters**: Chlorophyll-a, algae density
- **Spatial metadata**: Province, watershed, monitoring station locations


In [ ]:
# Load processed water quality data
df = pd.read_parquet('data/processed_water_quality.parquet')

print(f"Dataset shape: {df.shape}")
print(f"Date range: {df['monitoring_time'].min()} to {df['monitoring_time'].max()}")
print(f"Number of stations: {df['station_name'].nunique()}")
print(f"Number of provinces: {df['province'].nunique()}")
print(f"Number of watersheds: {df['watershed'].nunique()}")

# Display basic statistics
print("\nDataset Overview:")
print(df.info())

# Check for missing values
print("\nMissing Values:")
missing_data = df.isnull().sum()
missing_percentage = (missing_data / len(df)) * 100
missing_summary = pd.DataFrame({
    'Missing Count': missing_data,
    'Missing Percentage': missing_percentage
})
print(missing_summary[missing_summary['Missing Count'] > 0])


### 1.2 Data Quality Assessment

Assess data quality through statistical analysis and visualization of key water quality parameters.


In [ ]:
# Define key water quality parameters
water_quality_params = [
    'temperature', 'ph', 'dissolved_oxygen', 'conductivity', 'turbidity',
    'permanganate_index', 'ammonia_nitrogen', 'total_phosphorus', 'total_nitrogen',
    'chlorophyll_a', 'algae_density'
]

# Filter available parameters
available_params = [param for param in water_quality_params if param in df.columns]
print(f"Available water quality parameters: {available_params}")

# Calculate descriptive statistics
desc_stats = df[available_params].describe()
print("\nDescriptive Statistics:")
print(desc_stats.round(3))

# Create distribution plots
fig, axes = plt.subplots(3, 4, figsize=(20, 15))
axes = axes.flatten()

for i, param in enumerate(available_params):
    if i < len(axes):
        # Remove outliers for better visualization (using IQR method)
        Q1 = df[param].quantile(0.25)
        Q3 = df[param].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        filtered_data = df[(df[param] >= lower_bound) & (df[param] <= upper_bound)][param]
        
        axes[i].hist(filtered_data.dropna(), bins=50, alpha=0.7, edgecolor='black')
        axes[i].set_title(f'{param.replace("_", " ").title()}')
        axes[i].set_xlabel('Value')
        axes[i].set_ylabel('Frequency')
        axes[i].grid(True, alpha=0.3)

# Hide empty subplots
for i in range(len(available_params), len(axes)):
    axes[i].set_visible(False)

plt.tight_layout()
plt.show()

# Box plots for outlier detection
fig, ax = plt.subplots(figsize=(15, 8))
df[available_params].boxplot(ax=ax, rot=45)
ax.set_title('Water Quality Parameters - Box Plot Analysis')
ax.set_ylabel('Parameter Value')
plt.tight_layout()
plt.show()


## 2. Temporal Analysis and Trend Detection

### 2.1 Seasonal Patterns in Water Quality

Analyze seasonal variations in water quality parameters to identify natural cycles and potential anthropogenic influences.


In [ ]:
# Extract temporal features
df['year'] = df['monitoring_time'].dt.year
df['month'] = df['monitoring_time'].dt.month
df['day_of_year'] = df['monitoring_time'].dt.dayofyear
df['hour'] = df['monitoring_time'].dt.hour
df['season'] = df['month'].map({12: 'Winter', 1: 'Winter', 2: 'Winter',
                               3: 'Spring', 4: 'Spring', 5: 'Spring',
                               6: 'Summer', 7: 'Summer', 8: 'Summer',
                               9: 'Autumn', 10: 'Autumn', 11: 'Autumn'})

# Seasonal analysis for key parameters
key_params = ['temperature', 'dissolved_oxygen', 'ammonia_nitrogen', 'total_phosphorus', 'chlorophyll_a']
seasonal_data = df.groupby(['season', 'month'])[key_params].mean().reset_index()

# Create seasonal plots
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

for i, param in enumerate(key_params):
    if i < len(axes):
        # Seasonal box plot
        df.boxplot(column=param, by='season', ax=axes[i])
        axes[i].set_title(f'{param.replace("_", " ").title()} by Season')
        axes[i].set_xlabel('Season')
        axes[i].set_ylabel('Value')

# Hide empty subplot
axes[5].set_visible(False)

plt.tight_layout()
plt.show()

# Monthly trends
monthly_avg = df.groupby('month')[key_params].mean()

fig, ax = plt.subplots(figsize=(15, 8))
for param in key_params:
    ax.plot(monthly_avg.index, monthly_avg[param], marker='o', label=param.replace('_', ' ').title(), linewidth=2)

ax.set_title('Monthly Average Water Quality Parameters')
ax.set_xlabel('Month')
ax.set_ylabel('Average Value')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()


## 3. Principal Component Analysis (PCA)

### 3.1 Dimensionality Reduction and Pattern Discovery

Apply PCA to identify the main drivers of water quality variation and reduce dimensionality while preserving maximum variance.


In [ ]:
# Prepare data for PCA
pca_data = df[available_params].dropna()
print(f"Data for PCA: {pca_data.shape}")

# Standardize the data
scaler = StandardScaler()
pca_data_scaled = scaler.fit_transform(pca_data)

# Apply PCA
pca = PCA(n_components=min(10, len(available_params)))
pca_result = pca.fit_transform(pca_data_scaled)

# Explained variance analysis
explained_variance_ratio = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance_ratio)

print(f"\nExplained Variance Ratio:")
for i, (var_ratio, cum_var) in enumerate(zip(explained_variance_ratio, cumulative_variance)):
    print(f"PC{i+1}: {var_ratio:.3f} ({var_ratio*100:.1f}%) | Cumulative: {cum_var:.3f} ({cum_var*100:.1f}%)")

# Plot explained variance
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Individual explained variance
ax1.bar(range(1, len(explained_variance_ratio) + 1), explained_variance_ratio, 
        alpha=0.7, color='skyblue', edgecolor='navy')
ax1.set_title('Explained Variance by Principal Component')
ax1.set_xlabel('Principal Component')
ax1.set_ylabel('Explained Variance Ratio')
ax1.grid(True, alpha=0.3)

# Cumulative explained variance
ax2.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, 
         marker='o', linewidth=2, markersize=8, color='red')
ax2.axhline(y=0.95, color='green', linestyle='--', alpha=0.7, label='95% Variance')
ax2.axhline(y=0.80, color='orange', linestyle='--', alpha=0.7, label='80% Variance')
ax2.set_title('Cumulative Explained Variance')
ax2.set_xlabel('Number of Components')
ax2.set_ylabel('Cumulative Explained Variance Ratio')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 4. Granger Causality Analysis

### 4.1 Causal Relationships in Water Quality Parameters

Apply Granger causality tests to identify causal relationships between different water quality parameters, helping understand pollution propagation mechanisms.


In [ ]:
from statsmodels.tsa.stattools import grangercausalitytests

def perform_granger_test(data, cause_var, effect_var, max_lags=6):
    """Perform Granger causality test between two variables"""
    try:
        # Prepare data
        test_data = data[[effect_var, cause_var]].dropna()
        
        if len(test_data) < 30:  # Need sufficient data
            return None
        
        # Perform Granger test
        results = grangercausalitytests(test_data, maxlag=max_lags, verbose=False)
        
        # Extract p-values for each lag
        p_values = []
        for lag in range(1, max_lags + 1):
            if lag in results:
                p_val = results[lag][0]['ssr_ftest'][1]
                p_values.append(p_val)
        
        # Find best lag (minimum p-value)
        if p_values:
            best_lag = np.argmin(p_values) + 1
            best_p_value = min(p_values)
            
            return {
                'cause_var': cause_var,
                causes_var': effect_var,
                'best_lag': best_lag,
                'p_value': best_p_value,
                'significant': best_p_value < 0.05,
                'all_p_values': p_values
            }
        
        return None
    
    except Exception as e:
        print(f"Error in Granger test for {cause_var} -> {effect_var}: {e}")
        return None

# Define variable pairs for causality testing
causality_pairs = [
    ('ammonia_nitrogen', 'total_phosphorus'),
    ('total_phosphorus', 'chlorophyll_a'),
    ('total_nitrogen', 'chlorophyll_a'),
    ('temperature', 'dissolved_oxygen'),
    ('ph', 'ammonia_nitrogen'),
    ('turbidity', 'algae_density'),
    ('conductivity', 'total_phosphorus')
]

# Perform Granger causality tests
granger_results = []

for station in df['station_name'].unique()[:5]:  # Limit to 5 stations for demo
    station_data = df[df['station_name'] == station].sort_values('monitoring_time')
    
    for cause_var, effect_var in causality_pairs:
        if cause_var in station_data.columns and effect_var in station_data.columns:
            result = perform_granger_test(station_data, cause_var, effect_var)
            if result:
                result['station'] = station
                granger_results.append(result)

granger_df = pd.DataFrame(granger_results)

# Summary of Granger causality results
if not granger_df.empty:
    print("Granger Causality Analysis Results:")
    print(f"Total tests performed: {len(granger_df)}")
    print(f"Significant relationships: {granger_df['significant'].sum()}")
    
    # Display significant results
    significant_results = granger_df[granger_df['significant']]
    if not significant_results.empty:
        print("\nSignificant Causal Relationships:")
        for _, row in significant_results.iterrows():
            print(f"{row['cause_var']} -> {row['effect_var']} (Station: {row['station']})")
            print(f"  Lag: {row['best_lag']}, P-value: {row['p_value']:.4f}")
    
    # Visualize p-values
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Create a heatmap of p-values
    causality_matrix = granger_df.groupby(['cause_var', 'effect_var'])['p_value'].mean().unstack()
    
    if not causality_matrix.empty:
        sns.heatmap(causality_matrix, annot=True, cmap='RdYlBu_r', 
                   center=0.05, vmin=0, vmax=1, ax=ax)
        ax.set_title('Granger Causality P-Values Heatmap')
        plt.tight_layout()
        plt.show()


In [ ]:
# Import lifecycle tracking modules
import sys
sys.path.append('../python')
from lifecycle.product_trace import SmartphoneLifecycleTracker, WaterQualityCorrelationAnalyzer

# Initialize lifecycle tracker
tracker = SmartphoneLifecycleTracker()

# Calculate total pollution per phone
total_pollution = tracker.calculate_total_pollution_per_phone()
print("Total pollution per smartphone:")
for pollutant, amount in total_pollution.items():
    print(f"  {pollutant}: {amount:.3f}")

# Estimate regional pollution impact (1 million phones)
regional_impact = tracker.estimate_regional_pollution_impact(production_volume=1000000)
print(f"\nRegional pollution impact (1M phones):")
for location, impact in regional_impact.items():
    print(f"\n{location}:")
    for pollutant, amount in impact['pollutants'].items():
        if amount > 0:
            print(f"  {pollutant}: {amount:,.1f} kg")

# Analyze correlations with water quality
analyzer = WaterQualityCorrelationAnalyzer(tracker)
correlations = analyzer.analyze_temporal_correlations(df)

print(f"\nFound {len(correlations)} correlations between manufacturing and water quality")
significant_correlations = correlations[correlations['significant']]
print(f"Significant correlations: {len(significant_correlations)}")

# Identify pollution hotspots
hotspots = analyzer.identify_pollution_hotspots(df)
print(f"\nPollution hotspots identified: {len(hotspots)}")
for hotspot in hotspots[:3]:  # Top 3
    print(f"  {hotspot['zone']}: avg_correlation={hotspot['avg_correlation']:.3f}")

# Generate network data for visualization
network_data = tracker.create_supply_chain_network_data()
print(f"\nSupply chain network: {network_data['metadata']['total_nodes']} nodes, {network_data['metadata']['total_links']} links")


## 6. Advanced Machine Learning Models

### 6.1 LSTM Time Series Forecasting

Implement deep learning models for multi-step ahead forecasting of water quality parameters.


In [ ]:
# Import forecasting modules
from ml.forecasting.lstm_model import WaterQualityLSTM, MultiTargetLSTM
from ml.forecasting.prophet_model import WaterQualityProphet

# Prepare data for forecasting
forecast_data = df[df['station_name'].isin(df['station_name'].unique()[:3])].copy()  # Use 3 stations for demo
forecast_data = forecast_data.sort_values(['station_name', 'monitoring_time'])

# Define target variables for forecasting
target_variables = ['ph', 'dissolved_oxygen', 'ammonia_nitrogen', 'total_phosphorus']

# Train LSTM model
print("Training LSTM model...")
lstm_model = WaterQualityLSTM(sequence_length=24, prediction_horizon=6)
lstm_model.train(forecast_data, target_variables, validation_split=0.2, epochs=50)

# Make predictions
predictions = lstm_model.predict(forecast_data)
print(f"Generated predictions for {len(predictions)} stations")

# Train Prophet model
print("\nTraining Prophet model...")
prophet_model = WaterQualityProphet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=False,
    changepoint_prior_scale=0.05
)

prophet_model.train_multi_station(forecast_data, target_variables)
prophet_forecasts = prophet_model.forecast(forecast_data, target_variables, periods=168, freq='4H')

print(f"Prophet forecasts generated for {len(prophet_forecasts)} variables")

# Evaluate models
print("\nEvaluating model performance...")
lstm_metrics = lstm_model.evaluate(forecast_data, test_split=0.2)
print("LSTM Model Metrics:")
for metric, value in lstm_metrics.items():
    print(f"  {metric}: {value:.4f}")

prophet_metrics = prophet_model.evaluate_all_models(forecast_data, target_variables, test_days=30)
print(f"\nProphet Model Metrics:")
for target, station_metrics in prophet_metrics.items():
    if station_metrics:
        avg_rmse = np.mean([m['rmse'] for m in station_metrics.values()])
        print(f"  {target}: Average RMSE = {avg_rmse:.4f}")


## 7. Network Analysis and Community Detection

### 7.1 Pollution Propagation Network

Build network models to understand pollution propagation patterns and identify critical monitoring stations.


In [ ]:
# Create pollution correlation network
def build_pollution_network(df, threshold=0.5):
    """Build network based on pollution correlations"""
    G = nx.Graph()
    
    # Calculate correlations between stations for each parameter
    stations = df['station_name'].unique()[:10]  # Limit for demo
    
    for param in key_params:
        if param in df.columns:
            param_data = df[df['station_name'].isin(stations)].pivot_table(
                index='monitoring_time', 
                columns='station_name', 
                values=param
            ).corr()
            
            # Add edges for high correlations
            for i, station1 in enumerate(param_data.index):
                for j, station2 in enumerate(param_data.columns):
                    if i < j and not pd.isna(param_data.loc[station1, station2]):
                        corr = abs(param_data.loc[station1, station2])
                        if corr > threshold:
                            if G.has_edge(station1, station2):
                                G[station1][station2]['weight'] += corr
                                G[station1][station2]['parameters'].append(param)
                            else:
                                G.add_edge(station1, station2, weight=corr, parameters=[param])
    
    return G

# Build network
pollution_network = build_pollution_network(df, threshold=0.3)

print(f"Network Statistics:")
print(f"  Nodes (stations): {pollution_network.number_of_nodes()}")
print(f"  Edges (correlations): {pollution_network.number_of_edges()}")

# Calculate network metrics
if pollution_network.number_of_nodes() > 0:
    # Degree centrality
    degree_centrality = nx.degree_centrality(pollution_network)
    betweenness_centrality = nx.betweenness_centrality(pollution_network)
    closeness_centrality = nx.closeness_centrality(pollution_network)
    
    # Community detection using Louvain algorithm
    try:
        import networkx.algorithms.community as nx_comm
        communities = nx_comm.louvain_communities(pollution_network)
        print(f"  Communities detected: {len(communities)}")
        
        # Identify most central stations
        top_stations = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:5]
        print(f"\nMost Central Stations (by degree):")
        for station, centrality in top_stations:
            print(f"  {station}: {centrality:.3f}")
            
    except ImportError:
        print("NetworkX community module not available")
    
    # Visualize network
    plt.figure(figsize=(15, 10))
    pos = nx.spring_layout(pollution_network, k=1, iterations=50)
    
    # Draw nodes with size based on degree centrality
    node_sizes = [degree_centrality[node] * 1000 for node in pollution_network.nodes()]
    nx.draw_networkx_nodes(pollution_network, pos, node_size=node_sizes, 
                          node_color='lightblue', alpha=0.7)
    
    # Draw edges with width based on correlation strength
    edges = pollution_network.edges()
    edge_weights = [pollution_network[u][v]['weight'] for u, v in edges]
    nx.draw_networkx_edges(pollution_network, pos, width=[w*3 for w in edge_weights], 
                          alpha=0.5, edge_color='gray')
    
    # Draw labels
    nx.draw_networkx_labels(pollution_network, pos, font_size=8)
    
    plt.title('Pollution Correlation Network\n(Node size = Degree centrality, Edge width = Correlation strength)')
    plt.axis('off')
    plt.tight_layout()
    plt.show()


## 8. Anomaly Detection and Outlier Analysis

### 8.1 Advanced Anomaly Detection Methods

Apply multiple anomaly detection techniques to identify unusual pollution events and potential environmental incidents.


In [ ]:
# Anomaly detection using multiple methods
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.cluster import DBSCAN

def detect_anomalies_comprehensive(data, contamination=0.1):
    """Apply multiple anomaly detection methods"""
    results = {}
    
    # Prepare data (remove non-numeric columns and handle missing values)
    numeric_data = data.select_dtypes(include=[np.number]).fillna(method='ffill').fillna(method='bfill')
    
    if len(numeric_data) < 100:  # Need sufficient data
        return results
    
    # Method 1: Isolation Forest
    iso_forest = IsolationForest(contamination=contamination, random_state=42)
    iso_anomalies = iso_forest.fit_predict(numeric_data)
    results['isolation_forest'] = iso_anomalies == -1
    
    # Method 2: One-Class SVM
    try:
        oc_svm = OneClassSVM(gamma='scale', nu=contamination)
        svm_anomalies = oc_svm.fit_predict(numeric_data)
        results['one_class_svm'] = svm_anomalies == -1
    except:
        results['one_class_svm'] = np.zeros(len(numeric_data), dtype=bool)
    
    # Method 3: Statistical outliers (Z-score)
    z_scores = np.abs(stats.zscore(numeric_data, axis=0, nan_policy='omit'))
    z_threshold = 3
    results['statistical_outliers'] = (z_scores > z_threshold).any(axis=1)
    
    # Method 4: DBSCAN clustering
    try:
        # Use PCA to reduce dimensionality for DBSCAN
        pca = PCA(n_components=min(5, numeric_data.shape[1]))
        data_pca = pca.fit_transform(numeric_data.fillna(numeric_data.mean()))
        
        dbscan = DBSCAN(eps=0.5, min_samples=5)
        cluster_labels = dbscan.fit_predict(data_pca)
        results['dbscan_outliers'] = cluster_labels == -1
    except:
        results['dbscan_outliers'] = np.zeros(len(numeric_data), dtype=bool)
    
    return results

# Apply anomaly detection to each station
anomaly_results = []
for station in df['station_name'].unique()[:5]:  # Limit for demo
    station_data = df[df['station_name'] == station].sort_values('monitoring_time')
    
    if len(station_data) > 100:
        anomalies = detect_anomalies_comprehensive(station_data)
        
        for method, anomaly_mask in anomalies.items():
            anomaly_count = anomaly_mask.sum()
            if anomaly_count > 0:
                anomaly_results.append({
                    'station': station,
                    'method': method,
                    'anomaly_count': anomaly_count,
                    'anomaly_rate': anomaly_count / len(anomaly_mask)
                })

anomaly_df = pd.DataFrame(anomaly_results)

if not anomaly_df.empty:
    print("Anomaly Detection Results:")
    print(anomaly_df.groupby('method')['anomaly_count'].agg(['sum', 'mean']))
    
    # Visualize anomaly detection results
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    axes = axes.flatten()
    
    methods = anomaly_df['method'].unique()
    for i, method in enumerate(methods[:4]):
        if i < len(axes):
            method_data = anomaly_df[anomaly_df['method'] == method]
            axes[i].bar(method_data['station'], method_data['anomaly_count'])
            axes[i].set_title(f'Anomalies Detected - {method.replace("_", " ").title()}')
            axes[i].set_xlabel('Station')
            axes[i].set_ylabel('Number of Anomalies')
            axes[i].tick_params(axis='x', rotation=45)
    
    # Hide empty subplots
    for i in range(len(methods), len(axes)):
        axes[i].set_visible(False)
    
    plt.tight_layout()
    plt.show()


## 9. Conclusions and Policy Recommendations

### 9.1 Key Findings Summary

Based on our comprehensive analysis of water quality data, machine learning models, and product lifecycle assessment, we present the following key findings and recommendations.


In [ ]:
# Generate comprehensive summary of findings
print("="*80)
print("INDUSTRIAL POLLUTION INTELLIGENCE - KEY FINDINGS SUMMARY")
print("="*80)

# 1. Data Overview
print(f"\n1. DATA OVERVIEW:")
print(f"   • Total monitoring stations: {df['station_name'].nunique()}")
print(f"   • Total measurements: {len(df):,}")
print(f"   • Date range: {df['monitoring_time'].min().strftime('%Y-%m-%d')} to {df['monitoring_time'].max().strftime('%Y-%m-%d')}")
print(f"   • Geographic coverage: {df['province'].nunique()} provinces, {df['watershed'].nunique()} watersheds")

# 2. Water Quality Status
if 'water_quality_grade' in df.columns:
    grade_dist = df['water_quality_grade'].value_counts()
    print(f"\n2. WATER QUALITY STATUS:")
    for grade, count in grade_dist.head().items():
        percentage = (count / len(df)) * 100
        print(f"   • Grade {grade}: {count:,} measurements ({percentage:.1f}%)")

# 3. Critical Pollutants
print(f"\n3. CRITICAL POLLUTANTS (by variability):")
param_variability = df[available_params].std().sort_values(ascending=False)
for param, std_val in param_variability.head(5).items():
    print(f"   • {param.replace('_', ' ').title()}: σ = {std_val:.3f}")

# 4. Seasonal Patterns
print(f"\n4. SEASONAL PATTERNS:")
if 'season' in df.columns:
    seasonal_summary = df.groupby('season')[key_params].mean()
    for param in key_params[:3]:  # Top 3 parameters
        if param in seasonal_summary.columns:
            winter_val = seasonal_summary.loc['Winter', param]
            summer_val = seasonal_summary.loc['Summer', param]
            change = ((summer_val - winter_val) / winter_val) * 100
            print(f"   • {param.replace('_', ' ').title()}: {change:+.1f}% change (Winter → Summer)")

# 5. Forecasting Performance
if 'lstm_metrics' in locals():
    print(f"\n5. FORECASTING PERFORMANCE:")
    for metric, value in lstm_metrics.items():
        print(f"   • {metric.replace('_', ' ').title()}: {value:.4f}")

# 6. Product Lifecycle Impact
print(f"\n6. PRODUCT LIFECYCLE IMPACT:")
if 'total_pollution' in locals():
    print(f"   • Total water usage per smartphone: {total_pollution['water_usage']:,} liters")
    print(f"   • Total energy consumption: {total_pollution['energy_consumption']:.1f} kWh")
    print(f"   • Total waste generation: {total_pollution['waste_generation']:.3f} kg")
    
    # Top pollutants
    pollutant_impacts = {k: v for k, v in total_pollution.items() 
                        if k not in ['water_usage', 'energy_consumption', 'waste_generation']}
    top_pollutant = max(pollutant_impacts.items(), key=lambda x: x[1])
    print(f"   • Highest pollutant: {top_pollutant[0]} ({top_pollutant[1]:.4f} kg per phone)")

# 7. Network Analysis
if 'pollution_network' in locals() and pollution_network.number_of_nodes() > 0:
    print(f"\n7. NETWORK ANALYSIS:")
    print(f"   • Stations in network: {pollution_network.number_of_nodes()}")
    print(f"   • Correlation connections: {pollution_network.number_of_edges()}")
    avg_degree = sum(dict(pollution_network.degree()).values()) / pollution_network.number_of_nodes()
    print(f"   • Average connections per station: {avg_degree:.1f}")

# 8. Anomaly Detection
if not anomaly_df.empty:
    print(f"\n8. ANOMALY DETECTION:")
    total_anomalies = anomaly_df['anomaly_count'].sum()
    print(f"   • Total anomalies detected: {total_anomalies:,}")
    best_method = anomaly_df.groupby('method')['anomaly_count'].sum().idxmax()
    print(f"   • Most effective method: {best_method.replace('_', ' ').title()}")

print("\n" + "="*80)
print("RECOMMENDATIONS:")
print("="*80)

recommendations = [
    "1. MONITORING NETWORK OPTIMIZATION:",
    "   • Deploy additional sensors in high-correlation zones identified by network analysis",
    "   • Implement real-time anomaly detection using LSTM models",
    "   • Focus monitoring on critical pollutants with high variability",
    "",
    "2. INDUSTRIAL REGULATION:",
    "   • Target smartphone manufacturing zones for stricter pollution controls",
    "   • Implement closed-loop water systems in electronics manufacturing",
    "   • Mandate pollution tracking across supply chains",
    "",
    "3. PREDICTIVE MANAGEMENT:",
    "   • Use ensemble forecasting models for early warning systems",
    "   • Implement seasonal adjustment protocols based on temporal patterns",
    "   • Develop pollution propagation models for upstream-downstream management",
    "",
    "4. POLICY INTERVENTIONS:",
    "   • Focus on eutrophication control (ammonia nitrogen, total phosphorus)",
    "   • Implement targeted interventions in identified pollution hotspots",
    "   • Develop circular economy frameworks for product lifecycle management"
]

for rec in recommendations:
    print(rec)

print("\n" + "="*80)
print("ANALYSIS COMPLETED SUCCESSFULLY")
print("="*80)
